In [1]:
import numpy as np
import torch
import torchvision.datasets
import torchvision.models
import torchvision.transforms
from datetime import datetime
from tqdm import tqdm, trange
import warnings
warnings.filterwarnings("ignore")

In [2]:
from energy_rbm import *
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import numpy as np
# np.seterr(divide='ignore', invalid='ignore')


import re
from time import sleep

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Apr 15 13:56:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0

if CUDA:
    device='cuda'
else:
    device='cpu'
torch.cuda.is_available()
device='cpu'

In [5]:

n_hid=4
model0=torch.load('2021-01-28_n_hid={n_hid}_vol=12000_epoch=5000'.format(n_hid=n_hid), map_location=device).float()

def binary_to_decimal(list0):
    value=0
    for i in range(len(list0)):
        value+=list0[-i-1]*2**(i)
    return value

def decimal_to_binary(integer):
    string=bin(integer)[2:]
    list0=[float(d) for d in string]
    while len(list0)<n_hid:
        list0=[0.]+list0
    return torch.tensor([list0])

def check_energy(model0, n_sample=10000, step_eq=500):
    n_vis=len(model0.v[0]); n_hid=len(model0.h[0])
    state_E_dict={}
    hidden_state_list=[]
    for i in range(2**n_hid):
        state_E_dict[str(i)]=0
    # for i in range(2**n_hid):
    for i in [0]:
        hidden_state_list.append(decimal_to_binary(i))
    # for i in tqdm(range(len(hidden_state_list))):
    for i in range(len(hidden_state_list)):
        h_sample=hidden_state_list[i].to(device).detach()
        for j in range(step_eq):
            v_sample=model0.hidden_to_visible(h_sample).detach()
            v_sample=v_sample.bernoulli().detach()
            h_sample=model0.visible_to_hidden(v_sample).detach()
            h_sample=h_sample.bernoulli().detach()
        v_sample=model0.hidden_to_visible(h_sample).detach()
        v_sample=v_sample.bernoulli().detach()
        for n in tqdm(range(n_sample)):
        # for n in range(n_sample):
            h_sample=model0.visible_to_hidden(v_sample).detach()
            h_sample=h_sample.bernoulli().detach()
            v_sample=model0.hidden_to_visible(h_sample).detach()
            v_sample=v_sample.bernoulli().detach()
            # state_E_dict[str(binary_to_decimal(h_sample.view(4).int().detach().numpy()))].append(model0.energy2(v_sample, h_sample).detach().numpy()[0][0].astype(int))
            state_E_dict[str(binary_to_decimal(h_sample.view(n_hid).cpu().int().detach().numpy()))]+=1
#     print(state_E_dict)
    with open('state_E_dict_n_hid={n_hid}_n_sample={n_sample}.pkl'.format(n_hid=n_hid, n_sample=n_sample), 'wb') as f:
        pkl.dump(state_E_dict, f)


In [ ]:
check_energy(model0, n_sample=100000000, step_eq=1)

 18%|█▊        | 17949982/100000000 [50:48<3:54:34, 5829.72it/s]